# HW4 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Mar 9 2018

In [ ]:
# Make plots inline
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
# set_matplotlib_formats('png', 'pdf')
# set_matplotlib_formats('png')

# import modules for plotting and data analysis
import matplotlib.pyplot as plt

## 1. 

> Use the ATT dataset http://www.cl.cam.ac.uk/research/dtg/attarchive/pub/data/att faces.zip. to compute subspaces for the PCA and LDA methods. Provide illustration of the respective 1st, 2nd and 3rd eigenvectors. Compute the recognition rates for the test-set. Report

* Correct classification
* Incorrect classification

> Provide at least one suggestion for how you might improve performance of the system

In [ ]:
import numpy as np
import pandas as pd
import pathlib

from collections import OrderedDict
from PIL import Image

In [ ]:
def read_data(data_dir_path):
    rows = []
    height, width = 112, 92
    flattened = height * width # 10304
    X_keys = ["X_" + str(i) for i in range(flattened)]
    paths = pathlib.Path(data_dir_path).glob('**/*.pgm')
    for path in paths:
        im = Image.open(str(path))
        X_values = np.array(im).flatten()
        y_value = path.parent.name
        row = OrderedDict(zip(X_keys, X_values))
        row['y'] = y_value
        rows.append(row)

    df = pd.DataFrame(rows)
    return df

In [ ]:
data_dir_path = 'data/att_faces/'

In [ ]:
df = read_data(data_dir_path)
X = df.drop('y',axis=1)
y = df['y']

In [ ]:
plt.figure(figsize=(12,10))
numbers = [0,1,398,399]
for i in range(4):
    im = X.loc[numbers[i]].values.reshape(112, 92)
    label = y.loc[numbers[i]]
    plt.subplot(2, 2, i+1)
    plt.imshow(im, cmap='gray')
    plt.axis("off")
    plt.title(label)
plt.tight_layout()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import math
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(),
                                                    test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=X_train.shape[1])
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
print('X_std_pca.shape:', X_train_pca.shape)
print('pca.components_:', pca.components_.shape)

In [ ]:
plt.figure(figsize=(12,10))
for i in range(20):
    eigenvector = pca.components_[i].reshape(112, 92)
    plt.subplot(4, 5, i+1)
    plt.imshow(eigenvector, cmap='gray')
    plt.title("Eigenvector: {0}".format(i+1))
    plt.axis("off")
plt.tight_layout()

## 2.

> In robotics sound source localization is a frequent challenge. In the file http://www.hichristensen.com/demo-delay.wav estimate the delay between the two sound channels using FFT. provide an explanation of how you computes the delay between the two channels.

> For both questions provide a description of the approach adopted, the associated code and a description of your results.

To determine the delay between two signals, we’re essentially searching for the phase offset between them. This can be done by checking the cross correlation, or convolution one signal with the other. More specifically, this can be done using the FFT by reformulating the convolution operation in the time domain into a single multiplication in the fourier domain. I.e. using the convolution theorem:

$$
\mathcal{F}\{f*g\}=\mathcal{F}\{f\} \cdot \mathcal{F}\{g\}
$$

Thus, we can find the maximum response of the convolution between the two signals, i.e the counter shift rendering the signals in maximum alignment, by taking the argmax over the discrete inverse fourier transform of the product. From this we can determine $k$ in $k \cdot dt$, where $dt$ is the sample period for the discrete signal, $k$ is the number of samples and thus $k \cdot dt$ is the total phase shift in time.

First we import some functions for reading in the WAV file, and calculating the forward and infevers FFT.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

from scipy import signal
from scipy.fftpack import fft
from scipy.fftpack import ifft
from scipy.io import wavfile

We then read in the file and split the two channels. From the stereo format of the file, we'll note the left channel to be the first, and the right channel to be the second, as is conventional in sound engineering and file formats.

In [ ]:
fs, data = wavfile.read('data/demo-delay.wav')
L = data.T[0]
R = data.T[1]

From listening to the audio recording using a stereo headset, we can tell from the delay in arrival using our directional sense of hearing that the sound source is seems to originate from the left side from the perspective listener. If we take a look at the start of the signal closely, we can clearly see the subtle time shift with the right channel lagging behind the left.

This makes intuitive sense and matches with our sense of direction given that the time of arrival for a sound source on our left side would sound sooner in our left ear than our right. This ability for sub microsecond phase shift detection is what enables most binaural animals to localize sound sources, often coupled with servoing of the head to more accurately gauge bearing when sensing the change in phase shift over time and orientation.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(np.array_split(L, 500)[0], 'b', label='Left')
ax.plot(np.array_split(R, 500)[0], 'r', label='Right')
ax.set_xlabel('Samples (Discrete Time)')
ax.set_ylabel('Amplitide (Microphone Signal)')
ax.legend()
plt.show()

Before processing and comparing the audio samples, it is generally a good idea to remove the DC component, as we don't yet exactly know how else the two signals may differ. Here we'll do so by removing the mean average respectively from both channels.

For microphone arrays separated by larger distances or some insulation that would otherwise result in greater signal attenuation in one recorded channel that the other, normalizing each channel by its standard deviation may also be a good idea. As it seem from the figure above that the two signals are of the exact same amplitude, the introduction of smaller floating point computation does not seem necessary in this case.

In [ ]:
L = L - L.mean()
# L = L/L.std()
R = R - R.mean()
# R = R/R.std()

We'll use numpy to compute the N-dimensional discrete Fourier Transform for our real input signal. However, to ensure our fft computation is optimized for radix {2, 3, 4, 5}, we must find the next composite of the prime factors 2, 3, and 5 which are greater than or equal to target to pad with zeros, i.e. our maximum discrete sample size of our convolution operation, $|L| + |R| - 1$. This is also known as 5-smooth numbers, regular numbers, or Hamming numbers. We also compute the slice needed for later when shaving off the excess padding from our inverse operation.

Source: https://github.com/scipy/scipy/pull/3144

In [ ]:
s1 = np.array(L.shape)
s2 = np.array(R.shape)

shape = s1 + s2 - 1
fshape = [scipy.fftpack.helper.next_fast_len(int(d)) for d in shape]
fslice = tuple([slice(0, int(sz)) for sz in shape])

print("s1: ", s1)
print("s2: ", s2)
print("fshape: ", fshape)
print("fslice: ", fslice)

Now we can finally compute the FFT for the left and right channels, multiply them in the fourier domain, then recover the convolution by taking the reverse FFT of the product. Note that in order to compute the convolution correctly, we must first reverse the kernel, or the right channel in this case, before computing its FFT.

In [ ]:
sp1 = np.fft.rfftn(L, fshape)
sp2 = np.fft.rfftn(np.flip(R, axis=0), fshape)
ret = (np.fft.irfftn(sp1 * sp2, fshape)[fslice].copy())

We can now plot the response. As shown below, we see the response grow from zero to some nominal noise floor as the kernel convolves over the left channel. From the sharp spike in response just past 200k on the x axis, we can tell that the signals are indeed quite similar, as they best aling at a narrow range in phase offset.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(ret)
# ax.set_yscale("log", nonposy='clip')
plt.show()

We can examine the response closer and see the residual harmonics in the sliding convictions, given the arbitrary waveform my happen to coincide to lesser extents at period lengths of other carrier frequencies of the audible voice sample.

In [ ]:
nsamples = L.size

ax = plt.subplot(111)
ax.plot(ret[nsamples-1000:nsamples+1000])
# ax.set_yscale("log", nonposy='clip')
plt.show()

Now, if we use the sample frequency as declared in the metadata of the file, $44.1kHz$ in this case, to derive the sample period $dt$, we can multiple this with $k$ to determine the time shift. Note that we can't really speak of the phase angle, as we are working with arbitrary waveforms that are not composed of a single frequency, thus a single angle would not make sense.

In [ ]:
dt = np.arange(1-nsamples, nsamples)
recovered_sample_shift = dt[ret.argmax()]

f = 44.1e3
p = 1/f
recovered_time_shift = recovered_sample_shift * p

print("Recovered shift (samples): ", recovered_sample_shift)
print("Recovered shift (seconds): ", recovered_time_shift)

We can do a simple sanity check by multiplying this time by the speed of sound, $343 m/s$  in ambient air temperature to square a 'distance' between the two phased array microphones. We find this to be around $\sim17cm$, which is an approximately on the scale for human ear separation, thus why the acoustic delay sound naturally from the left for us.

In [ ]:
print("Array Displacement", 343*recovered_time_shift)

We can also check this by reversing the shift on the right channel as to realign with the left channel waveform. From the figure below, we see that $k=-22$ does indeed bring the two waveform back info perfect synchrony.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(np.array_split(L[0:], 1000)[0][100:110],'bv-', label='Left')
ax.plot(np.array_split(R[21:], 1000)[0][100:110],'g-', label='Right (-21)')
ax.plot(np.array_split(R[23:], 1000)[0][100:110],'y-', label='Right (-23)')
ax.plot(np.array_split(R[22:], 1000)[0][100:110],'ro--', label='Right (-22)')
ax.set_xlabel('Samples (Discrete Time)')
ax.set_ylabel('Amplitide (Microphone Signal)')
ax.legend()
plt.show()